In [1]:
import sys
sys.path.append("..")
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import sys
from scipy import optimize
from torch.utils.data import DataLoader, Dataset
from data_loaders import *
import missing_process.missing_method as missing_method
from missing_process.block_rules import *
import json
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [2]:
real_datalist = ["banknote","concrete_compression",
            "wine_quality_white","wine_quality_red",
            "california","climate_model_crashes",
            "connectionist_bench_sonar","qsar_biodegradation",
            "yeast","yacht_hydrodynamics"
            ]
#real_datalist = ["yacht_hydrodynamics"]

syn_datalist = ["syn1"]


missingtypelist = ["quantile","diffuse","logistic"]
#missingtypelist = ["diffuse"]

In [6]:
def create_missing(missingtypelist,datalist_name):
    '''
    Create Mask npy
    Save missing 
    '''
    

    if datalist_name == "real":
        datalist = real_datalist
    elif datalist_name == "syn":
        datalist = syn_datalist

    for missingtype in missingtypelist:
        if missingtype == "logistic":
            missing_rule = load_json_file("missing_rate.json")
        elif missingtype == "diffuse":
            missing_rule = load_json_file("diffuse_ratio.json")
        elif missingtype == "quantile":
            missing_rule = load_json_file("quantile_full.json")


        missing_rate_d = {}

        for dataname in datalist:
            directory_path = f"../datasets/{dataname}"    
            norm_values = np.load(f'{directory_path}/{dataname}_norm.npy')

            print(dataname)
            missing_rate_d[dataname] = []

                # Check if the directory exists
            if not os.path.exists(f'{directory_path}/{missingtype}'):
                os.makedirs(f'{directory_path}/{missingtype}')
            else:
                pass
            
            for rule_name in missing_rule:

                rule = missing_rule[rule_name]
                # try:
                #     create_mask(norm_values,missingtype,rule)
                # except:
                #     print(dataname,missingtype)
                
                observed_masks = create_mask(norm_values,missingtype,rule)

                np.save(f'{directory_path}/{missingtype}/{rule_name}.npy', observed_masks)


                missing_rate = 1 - np.count_nonzero(observed_masks) / observed_masks.size
                #print(rule_name,missing_rate)

                missing_rate_d[dataname].append(missing_rate)

        df = pd.DataFrame.from_dict(missing_rate_d)
        df.index = [rule_name for rule_name in missing_rule]
        
        df.to_csv(f"../datasets/{datalist_name}_{missingtype}_missing_rate.csv")

In [8]:
create_missing(missingtypelist,"real")

banknote
concrete_compression
wine_quality_white


C:\Users\Epochoid\AppData\Local\Temp\ipykernel_37608\2080298773.py:15: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  masks = np.array(~np.isnan(Xnan), dtype=np.float)


wine_quality_red
california
climate_model_crashes
connectionist_bench_sonar
qsar_biodegradation
yeast
yacht_hydrodynamics
banknote
concrete_compression
wine_quality_white
wine_quality_red
california
climate_model_crashes
connectionist_bench_sonar
qsar_biodegradation
yeast
yacht_hydrodynamics
banknote
concrete_compression
wine_quality_white
wine_quality_red
california
climate_model_crashes
connectionist_bench_sonar
qsar_biodegradation
yeast
yacht_hydrodynamics


In [9]:
def create_mask(observed_values,missing_type = "MCAR",
                  missing_para = None):

    observed_masks = ~np.isnan(observed_values.astype("float32"))
    masks = observed_masks.copy().astype("float32")


    "Need input origin dataset and parameters"
    if missing_type == "MCAR":
        masks = missing_method.MCAR(observed_values,0.2,masks)


    elif missing_type == "quantile":
        Xnan, Xz = missing_method.missing_by_range(observed_values, missing_para)
        masks = np.array(~np.isnan(Xnan), dtype=np.float)

    elif missing_type == "logistic":
        masks = missing_method.MNAR_mask_logistic(observed_values, missing_para)

    elif missing_type == "diffuse":
        masks =  missing_method.diffuse_mnar_single(observed_values, missing_para[0],missing_para[1])

    observed_masks = masks.astype(int)


    return observed_masks

